In [1]:
import torch
from torch import nn
from torch.nn import init
import numpy as np
import pandas as pd
import sys
sys.path.append('../../')
import ML as ml
import glob, os

In [4]:
data = pd.read_csv('earth_testset.csv', sep=',')
df = data[:10000]
df.status[df['status']==-1]=0

/home/kong123/anaconda3/envs/py37/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/kong123/anaconda3/envs/py37/lib/python3.7/site-packages/pandas/core/generic.py:8765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


In [5]:
all_features = df.iloc[:,:-1]
all_features

,u,e,a_p,inc
0,0.17,0.04,4.25,0.698132
1,0.13,0.30,2.55,1.745329
2,0.49,0.83,5.01,1.047198
3,0.12,0.62,2.60,2.967060
4,0.46,0.23,1.77,1.047198
...,...,...,...,...
9995,0.14,0.88,5.05,1.221730
9996,0.28,0.32,3.84,2.443461
9997,0.41,0.09,5.02,0.349066
9998,0.06,0.52,2.39,1.919862


In [6]:
s =set(all_features['inc'])
inc = list(s)
inc.sort()
for i in range(len(inc)):
    all_features.inc[all_features['inc']==inc[i]]='%d'%i

In [7]:
numeric_features = all_features.dtypes[all_features.dtypes != 'object'].index
all_features[numeric_features] = all_features[numeric_features].apply(
    lambda x: (x - x.mean()) / (x.std()))
# 标准化后，每个特征的均值变为0，所以可以直接用0来替换缺失值
all_features[numeric_features] = all_features[numeric_features].fillna(0)
#set(all_features['inc'])

In [8]:
all_features = pd.get_dummies(all_features, dummy_na=False)
all_features

,u,e,a_p,inc_0,inc_1,inc_10,inc_11,inc_12,inc_13,inc_14,...,inc_17,inc_18,inc_2,inc_3,inc_4,inc_5,inc_6,inc_7,inc_8,inc_9
0,-0.589782,-1.570858,0.389384,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
1,-0.868612,-0.668507,-0.909503,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1.640860,1.170901,0.970063,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,-0.938319,0.442079,-0.871300,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,1.431737,-0.911447,-1.505463,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.798904,1.344430,1.000625,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
9996,0.177001,-0.599095,0.076123,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9997,1.083200,-1.397329,0.977704,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
9998,-1.356565,0.095021,-1.031751,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
test_features = torch.tensor(all_features.values, dtype=torch.float)
test_labels = torch.tensor(df.status.values, dtype=torch.long)
batch_size = 512
testset = torch.utils.data.TensorDataset(test_features, test_labels)
test_iter = torch.utils.data.DataLoader(testset, batch_size, shuffle=True)

In [11]:
PATH = "./mini_net.pt"
model = torch.load(PATH)

In [12]:
test_acc = ml.eva_acc(test_iter, model)
test_acc

0.9691

In [13]:
ml.dnn_results(test_iter, model)#将dnn预测的结果导出

20

In [14]:
#读取结果
path = './'
file = glob.glob(os.path.join(path, "df**.csv"))
#合并结果
dl = []
for f in file:
    dl.append(pd.read_csv(f,))
df = pd.concat(dl)
df

,u,e,a_p,predictions,labels
0,-0.310951,1.552665,-0.282981,0,0
1,1.710568,-0.737918,-1.291529,1,1
2,-1.147442,-0.876742,-1.650633,0,0
3,0.386124,-1.084977,-1.016470,0,0
4,-0.729197,1.136195,-0.695569,0,0
...,...,...,...,...,...
507,0.943784,0.615608,-0.481634,0,0
508,-0.798904,0.754431,-1.314450,0,0
509,1.571152,-0.460272,0.526913,1,1
510,-0.798904,-0.668507,-0.053766,1,1


In [15]:
#保存
df.to_csv('mini_results.csv', index=False)
#删除多余的文件
for f in file:
    os.remove(f)

In [16]:
#模型评估
ml.dnn_eva(df)

Accuracy: 0.9691, Precision: 0.9648, Recall: 0.9845, F_1: 0.9745


(5914, 93, 216, 3777)